<a href="https://colab.research.google.com/github/AmaleshV/Medium-Posts/blob/master/TPU_Talos_hyperparameter_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Increase RAM

In [0]:
d=[]
while(1):
  d.append('1')

**Remove any pre-defined variables**

In [0]:
#remove all defined variables
%reset

# TPU + Talos Pipeline:

**Upload Kaggle JSON**

In [0]:
from google.colab import files
df = files.upload()

**Update kaggle api**

In [0]:
!pip install kaggle --upgrade

**Libraries Used**

In [0]:
#install required packages
!pip install git+https://github.com/autonomio/talos
!pip install xgboost
import xgboost as xgb
import pandas as pd
import numpy as np
import random
import sklearn
import re
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dropout, Dense
import keras.backend as K
import talos

# 1. Downloading the datasets from API calls**

In [0]:
%%time
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download --force ieee-fraud-detection

# 2.Pre-Processing and Data Wrangling:

In [0]:
%%time
import dask.dataframe as dd
train_identity = dd.read_csv('train_identity.csv')
train_transaction = dd.read_csv('train_transaction.csv')
test_identity = dd.read_csv('test_identity.csv')
test_transaction = dd.read_csv('test_transaction.csv')
sub = dd.read_csv('sample_submission.csv')
# let's combine the data and work with the whole dataset
train = dd.merge(train_transaction, train_identity, on='TransactionID', how='left')
test = dd.merge(test_transaction, test_identity, on='TransactionID', how='left')

In [0]:
#convert to pandas
train = train.compute()
test = test.compute()

In [0]:
#Data Wrangling
df=train
df=df.sample(frac=1,random_state=123) #shuffle data for memory bias
#from sklearn import preprocessing
# Identify which variables can be numeric and which factor (<32 uniq levels)
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
cat_cols= [col for col in df.columns if (df[col].dtypes==object) & (df[col].nunique() > 32)]
print(cat_cols)
df= df.drop(cat_cols, axis=1)
#df= df.loc[:,(df.dtypes== 'int64') | (df.dtypes==float) |((df.dtypes==object) & (df.nunique() < 32))]
# Select only columns with less than 32 unique values if object or select numeric variables
df_wrangled=df
df_wrangled=train
if 'isFraud' in df_wrangled:
        df_wrangled = df_wrangled.loc[df_wrangled['isFraud'].notnull(),:] #pick only those alerts with target variable defined
print(df_wrangled.head())
# # Train Test Split
# set x and y variables
if 'isFraud' in df_wrangled:
    y = df.isFraud
    x= df.loc[:, df.columns != 'isFraud']
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.1, random_state = 123)

In [0]:
# # Training Data preprocessing
# Eliminate automatically variables with more than 20% of missingness
xTrain = xTrain[xTrain.columns[xTrain.isnull().mean() < 0.2]]
# fill missing values with mean column values for numeric features
#xTrain.select_dtypes(include=numerics) =  xTrain.select_dtypes(include=numerics).fillna(xTrain.mean(), inplace=True)
#xTrain._get_numeric_data().fillna(xTrain.mean(), inplace=True)
cols= xTrain.columns
colnames_numerics_only = xTrain.select_dtypes(include=np.number).columns.tolist()
colnames_numerics_only
xTrain[colnames_numerics_only] = xTrain[colnames_numerics_only].fillna(xTrain[colnames_numerics_only].mean())
# impute categorical variables with mode
#cols = xTrain.select_dtypes(include= ).columns
cat_cols= list(set(cols) - set(colnames_numerics_only))
#xTrain[cat_cols] = xTrain.loc[:,xTrain.columns.isin(cat_cols) & (xTrain.isna().any())].fillna(xTrain[cat_cols].mode().iloc[0])
xTrain[cat_cols] = xTrain[cat_cols].fillna(xTrain[cat_cols].mode().iloc[0])
train_cols = xTrain.columns
#xTrain.loc[:,(xTrain.dtypes==object)] = xTrain.loc[:,(xTrain.dtypes==object)].fillna(xTrain.mode())
# get categorical features index
xTrain.info()

# 3. Feature Engineering and Feature Selection:

In [0]:
%%time
##change to GPU before running this code chunk and save the final_selected_cols as binary columns object so that you can use feature for later deep learning model
xTrain_dummy = pd.get_dummies(xTrain, prefix_sep='_', drop_first=True) #create dummies
xgb_cols =xTrain_dummy.columns
xTrain_xgb=xTrain_dummy
yTrain_xgb = yTrain
from sklearn.feature_selection import SelectFromModel
from xgboost import XGBClassifier
xgbc=XGBClassifier(n_estimators=500,verbose=1,tree_method='gpu_hist',random_state=123)
embeded_xgb_selector = SelectFromModel(xgbc)
embeded_xgb_selector.fit(xTrain_xgb, yTrain_xgb)
embeded_xgb_support = embeded_xgb_selector.get_support()
embeded_xgb_feature = xTrain_xgb.loc[:,embeded_xgb_support].columns.tolist()
final_selected_cols=embeded_xgb_feature
final_tr=xTrain_xgb[embeded_xgb_feature]

In [0]:
### Test Data preprocessing ###
# Eliminate automatically variables with more than 20% of missingness
xTest = xTest[xTest.columns[xTest.isnull().mean() < 0.2]]
# fill missing values with mean column values for numeric features
test_cols= xTest.columns
num_cols = xTest.select_dtypes(include=np.number).columns
xTest[num_cols] = xTest[num_cols].fillna(xTest[num_cols].mean())

# impute categorical variables with mode
#cols = xTest.select_dtypes(include= ).columns
cat_cols= list(set(test_cols) - set(num_cols))
#xTest[cat_cols] = xTest.loc[:,xTest.columns.isin(cat_cols) & (xTest.isna().any())].fillna(xTest[cat_cols].mode().iloc[0])
xTest.loc[:,cat_cols] = xTest[cat_cols].fillna(xTest[cat_cols].mode().iloc[0])

# Select only those features which are there in training #
#train_cols = xTrain.columns
xTest = xTest[train_cols] 



# Dummify categorical vars
xTest_dummy = pd.get_dummies(xTest, prefix_sep='_', drop_first=True)

##missing columns levels train and test
missing_levels_cols= list(set(xTrain_dummy.columns) - set(xTest_dummy.columns))


for c in missing_levels_cols:
    xTest_dummy[c]=0


# Select only those variables in final_tr
final_ts =xTest_dummy[final_selected_cols]

# 4. Scaling the Data:

In [0]:
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
# fit and transform only on training data  
final_tr_scaled = scaler.fit_transform(final_tr)  
### Apply on test data ###
final_ts_scaled = scaler.transform(final_ts) 

# 5. TPU Initialization:

In [0]:
import os
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_host(resolver.master())
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

# 6. Model Building:

In [0]:
def ieee_fraud_model(x_tr, y_tr, x_ts, y_ts, params):
# Specify a distributed strategy to use TPU
  with strategy.scope():

    model = Sequential()
    model.add(Dense(params['first_hidden_layer'], input_dim=final_tr.shape[1],
                            activation=params['activation1'],
                            kernel_initializer=params['kernel_initializer']))
    model.add(Dropout(params['dropout']))
    model.add(Dense(params['second_hidden_layer'], 
                            activation=params['activation2'],
                            use_bias=True))
    
    model.add(Dense(1, activation=params['last_activation']))
    
    model.compile(loss=params['losses'],
                            optimizer=params['optimizer'],
                            metrics=['binary_accuracy'])
  
  x_tr = np.asarray(x_tr).astype('float32')
  y_tr = np.asarray(y_tr).astype('float32').reshape((-1,1))
  x_ts = np.asarray(x_ts).astype('float32')
  y_ts = np.asarray(y_ts).astype('float32').reshape((-1,1))
  # Fit the Keras model on the dataset
  steps_per_epoch = int(np.ceil(x_tr.shape[0] / params['batch_size'])) - 1
  history = model.fit(x_tr, y_tr, 
                            validation_data=[x_ts, y_ts],
                            batch_size=params['batch_size'],
                            epochs=params['epochs'],
                          callbacks=[talos.utils.live()],
                            verbose=1)
  return history, model

# 7. Parameter Grid:

In [0]:
p = {'activation1':[ tf.nn.elu,tf.nn.relu],
     'activation2':[ tf.nn.elu,tf.nn.relu],
     'first_hidden_layer': [10,8],
     'second_hidden_layer': [6],
     'batch_size': [100, 400],
     'epochs': [30],
     'learn_rate': [0.1,0.01],
     'momentum': [0.2],
     'dropout': [0, 0.1],
     'weight_regulizer':[None],
     'emb_output_dims': [None],
     'optimizer': [ 'sgd','Adam', 'Nadam', 'RMSprop'],   
     'losses': ['binary_crossentropy','logcosh'],
     'last_activation': ['sigmoid'],
     'kernel_initializer':["GlorotUniform"]}

# 8. Talos hyperparameter scanning:

In [0]:
%%time
%matplotlib inline
from keras.models import load_model
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform
import os
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
  ty = talos.Scan(x=final_tr_scaled,
               y=yTrain,x_val=final_ts_scaled,y_val=yTest,
               model=ieee_fraud_model,
               params=p,
               experiment_name='Talos_metric_ba')

# 9. Prediction and submission file:

**Dataframe of all the models**

In [0]:
# use Scan object as input
analyze_object = talos.Analyze(ty)
# access the dataframe with the results
output_dataframe=analyze_object.data
# get the highest result for any metric
print(analyze_object.high('binary_accuracy'))
output_dataframe

**Prediction according to metric**

In [0]:
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
  talospred=talos.Predict(scan_object=ty)
  y_predi=talospred.predict(x=final_ts_scaled,metric='binary_accuracy',asc=True)

**Deploying the Best model**

In [0]:
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
  talos.Deploy(scan_object=ty, model_name='tal_model',metric='val_binary_accuracy')

**Restoring the best saved model from talos for predictions**

In [0]:
#restore the saved model
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
  tal_model = talos.Restore('tal_model.zip')

In [0]:
####submission prediction
### Test Data preprocessing ###
# Eliminate automatically variables with more than 20% of missingness
test = test[test.columns[test.isnull().mean() < 0.2]]
# fill missing values with mean column values for numeric features
test_cols= test.columns
num_cols = test.select_dtypes(include=np.number).columns
test[num_cols] = test[num_cols].fillna(test[num_cols].mean())

# impute categorical variables with mode
#cols = .select_dtypes(include= ).columns
cat_cols= list(set(test_cols) - set(num_cols))
#test[cat_cols] = test.loc[:,test.columns.isin(cat_cols) & (test.isna().any())].fillna(test[cat_cols].mode().iloc[0])
test.loc[:,cat_cols] = test[cat_cols].fillna(test[cat_cols].mode().iloc[0])

# Select only those features which are there in training #
#train_cols = xTrain.columns
test = test[train_cols] 



# Dummify categorical vars
test_dummy = pd.get_dummies(test, prefix_sep='_', drop_first=True)

##missing columns levels train and test
missing_levels_cols= list(set(xTrain_dummy.columns) - set(test_dummy.columns))


for c in missing_levels_cols:
    test_dummy[c]=0


# Select only those variables in final_tr
final_ts1 =test_dummy[final_selected_cols]
final_ts1_scaled = scaler.transform(final_ts1)

#predictions from the best talos model
ytest_seq=tal_model.model.predict(final_ts1_scaled)
### prepping submission file###
submission_seq= np.c_[test.TransactionID,ytest_seq]
submission_seq= pd.DataFrame(submission_seq)
submission_seq.columns= ['TransactionID','isFraud']
print(submission_seq)
submission_seq.to_csv('submission_seq.csv')
from google.colab import files
files.download('submission_seq.csv')

# For only Talos with CPU

In [0]:
def ieee_fraud_model(x_tr, y_tr, x_ts, y_ts, params):
# Specify a distributed strategy to use TPU
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Dense(params['first_hidden_layer'], input_dim=final_tr.shape[1],
                            activation=params['activation1'],
                            kernel_initializer=params['kernel_initializer']))
  model.add(tf.keras.layers.Dense(params['second_hidden_layer'], 
                            activation=params['activation2'],
                            kernel_initializer=params['kernel_initializer'] ,
                            use_bias=True))
  model.add(tf.keras.layers.Dense(params['third_hidden_layer'], 
                            activation=params['activation3'], 
                            use_bias=True,
                            kernel_initializer=params['kernel_initializer']))
  model.add(tf.keras.layers.Dense(1, activation=params['last_activation']))
  optimizer = tf.keras.optimizers.SGD(lr=params['learn_rate'], momentum=params['momentum'])
  model.compile(loss=params['losses'],
                          optimizer=optimizer,
                          metrics=['binary_accuracy'])
                      
  
  x_tr = np.asarray(x_tr).astype('float32')
  y_tr = np.asarray(y_tr).astype('float32').reshape((-1,1))
  x_ts = np.asarray(x_ts).astype('float32')
  y_ts = np.asarray(y_ts).astype('float32').reshape((-1,1))
  # Fit the Keras model on the dataset
  steps_per_epoch = int(np.ceil(x_tr.shape[0] / params['batch_size'])) - 1
  history = model.fit(x_tr, y_tr, 
                            validation_data=[x_ts, y_ts],
                            batch_size=params['batch_size'],
                            epochs=params['epochs'],steps_per_epoch=steps_per_epoch,
                          callbacks=[talos.utils.live()],
                            verbose=1)
  return history, model